In [1]:
import os
import tarfile
import json
import pandas as pd

In [89]:
#Set WD
os.chdir("/Users/andreaprenner/Desktop/Master_UvA/1. Semester/FundamentalsOfDS/Assignment1")

In [94]:
tar = tarfile.open("geotagged_tweets_20160812-0912.tar.gz")
files = tar.getmembers()

In [561]:
f = tar.extractfile(files[0]) 
data = f.readlines()

In [623]:
json.loads(data[0])

{'created_at': 'Fri Aug 12 10:04:00 +0000 2016',
 'id': 764039724818272256,
 'id_str': '764039724818272256',
 'text': '@theblaze @realDonaldTrump https://t.co/TY9DlZ584c',
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'in_reply_to_status_id': 764038820476051456,
 'in_reply_to_status_id_str': '764038820476051456',
 'in_reply_to_user_id': 10774652,
 'in_reply_to_user_id_str': '10774652',
 'in_reply_to_screen_name': 'theblaze',
 'user': {'id': 366636488,
  'id_str': '366636488',
  'name': 'GIL DUPUY',
  'screen_name': 'DUPUY77',
  'location': 'Miami',
  'url': 'http://ggm-dupuy.com',
  'description': "Fashion photographer, love action and adventure, care for the less fortunate, don't tolerate any kind of racism regardless of race or religion",
  'protected': False,
  'verified': False,
  'followers_count': 186,
  'friends_count': 446,
  'listed_count': 19,
  'favourites_count': 1708,
  'statuses_count': 17620,
  'cre

In [614]:
#Open questions: How to integrate user_mentions (entities - user_mentions) entries 
#(user_mentions is not in final data frame) and place (esp. coordinates) in data frame
#Filtering out bots
#Formatting text - removing stop words
data_formatted = []
for i in range(0, 100): 
    data_f = json.loads(data[i])
    user = data_f['user']
    user = pd.DataFrame([user])
    user = user.drop('id_str', axis=1)
    place = data_f['place']
    place = pd.DataFrame([place])
    place = place[['full_name', 'country_code', 'bounding_box']]
    unwanted = set(['user', 'place', 'entities'])
    for unwanted_key in unwanted: del data_f[unwanted_key]
    data_pre = pd.DataFrame([data_f])
    user = user.rename({'id':'user_id'}, axis=1)
    #user['id'] = data_f['id']
    if i == 0: 
        data_formatted = data_pre
        user_formatted = user
        place_formatted = place
    else:
        data_formatted = data_formatted.append(data_pre, ignore_index=True)
        user_formatted = user_formatted.append(user, ignore_index=True)
        place_formatted = place_formatted.append(place, ignore_index=True)

In [620]:
data_formatted = pd.concat([data_formatted, user_formatted, place_formatted], axis=1)

Index(['contributors', 'coordinates', 'created_at', 'extended_entities',
       'favorite_count', 'favorited', 'filter_level', 'geo', 'id', 'id_str',
       ...
       'screen_name', 'statuses_count', 'time_zone', 'url', 'user_id',
       'utc_offset', 'verified', 'full_name', 'country_code', 'bounding_box'],
      dtype='object', length=267)

**SENTIMENT ANALYSIS**

In [543]:
#Import NLTK 
import nltk
from nltk.corpus import stopwords
import random

In [544]:
#Training sentiment 
training_set = pd.read_csv("trainingandtestdata/training.1600000.processed.noemoticon.csv", 
                 encoding = 'latin-1',
                 names = ["Polarity", "Tweet ID","Date","Query","User","Text"])

In [545]:
#Filter out stop words in text column
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [546]:
#Create data that puts tweet texts together with sentiment score
tweets_and_polarity_scores = []
for i in range(0, len(training_set)):
    if training_set["Polarity"].iloc[i] == 0 or training_set["Polarity"].iloc[i] == 4: 
        tweet_text = training_set['Text'].iloc[i]
        tweet_words_in_list = [e.lower() for e in tweet_text.split()]
        tweet_words_in_list_wo_stopwords = [e for e in tweet_words_in_list if not e in stopwords]
        tweet_polarity_score = training_set["Polarity"].iloc[i]
        tweets_and_polarity_scores.append((tweet_words_in_list_wo_stopwords, tweet_polarity_score))
    else: 
        continue

In [547]:
random_sample = random.sample(tweets_and_polarity_scores, k = 1000)

In [548]:
def get_words_in_tweets(tweets):
    all_words= []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = list(wordlist.keys())
    return word_features

word_features = get_word_features(get_words_in_tweets(random_sample))

In [549]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features["contains(%s)" % word] = (word in document_words)
    return features

In [550]:
final_training_set = nltk.classify.apply_features(extract_features, random_sample)

In [551]:
classifier = nltk.NaiveBayesClassifier.train(final_training_set)

In [559]:
#Testing
tweet_positive = 'I dont like Larry'
classifier.classify(extract_features(tweet_positive.split()))
#classifier.show_most_informative_features(32)

0